The singature should be of float type ... DONE <br/>
Integrate FGSM ... DONE  <br/>
Visualize three adv samples (at the beginning of this notebook) ... DONE  <br/>
Integrate more attacks ... DONE  <br/>
Simply the code (create individually -> in a simple list) ... DONE <br/>
Try to enhance the ASRs ... DONE <br/>
Generate and store all possible attacks you could generate ... <br/>
Complete the generalization of the same type of adv attacks ...  <br/>
(if fail) -> try DRAW architecture ...  <br/>

In [1]:
import PI

meta_params = {
    'num_of_train_dataset': 1000,
    'num_of_test_dataset': 100,
    'is_flatten': False
}

PI = PI.PIInterface(meta_params)

import torch
import torch.nn as nn
import torch.optim as optim
from MNIST_models import *
# model = CNN()
# loss_func, optimizer = nn.CrossEntropyLoss(), torch.optim.Adam(model.parameters(), lr=1e-3)
# model = PI.train_model(model, loss_func, optimizer, 3)
# PI.set_model(model)
# print('train acc:', PI.eval_model('train'))
# print('test acc:', PI.eval_model('test'))
# print()

model = load_model('store/MNIST_CNN.pt')
PI.set_model(model)
print('train acc:', PI.eval_model('train'))
print('test acc:', PI.eval_model('test'))

Using TensorFlow backend.
//anaconda3/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'MNIST_models.CNN' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
//anaconda3/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
//anaconda3/lib/python3.7/site-packages/torch/serialization.py:453: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute o

train acc: 0.98
test acc: 0.99


In [2]:
# # visualize the benign input and corresponding adv samples 
# samples, labels = PI.train_dataset

# %matplotlib inline
# from matplotlib import pyplot as plt
# import numpy as np
# import attacker
# import time
# import warnings
# warnings.filterwarnings('ignore')


# # adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'L1PGD', 'L2PGD', 'LINFPGD', 'L2BI', 'LINFBI', 'ENL1', 'DDNL2', 'LBFGS', 'SP', 'LS', 'ST']
# adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
# print(len(adv_types))
# formatted_adv_types = [adv_type.ljust(6) for adv_type in adv_types]
# print('  '+' '.join(formatted_adv_types))

# figsize=(5, 5)
# fig=plt.figure(figsize=figsize)
# columns, num_of_rows = len(adv_types), 8

# for i in range(num_of_rows):
#     sample, label = samples[i], labels[i]
#     for col_index in range(columns):  
#         adv_type = adv_types[col_index]
#         if adv_type == 'None': 
#             plt_sample = sample 
#         else:
#             plt_sample = PI.generate_adv_img(sample, label, model, adv_type)
#             if plt_sample is None: 
#                 plt_sample = np.zeros((1, 28, 28)) 
            

#         fig.add_subplot(num_of_rows, columns, (col_index+1)+(i*columns))
#         plt.imshow(np.squeeze(plt_sample), cmap='gray')
#         plt.xticks([])
#         plt.yticks([])

# fig.savefig('out.png')

In [3]:
import pickle

# STORE
# adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
# adv_types = ['CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
# for adv_type in adv_types:
#     if adv_type == 'None': 
#         fn_name='store/normal.txt'
#         set_of_signatures = PI.generate_signatures() 
#     else: 
#         fn_name='store/'+adv_type+'.txt'
#         set_of_signatures = PI.generate_signatures(adv_type=adv_type)
    
#     print(adv_type, len(set_of_signatures))
#     with open(fn_name, "wb") as fp:   
#         pickle.dump(set_of_signatures, fp)
    
    
# LOAD 
# adv_types = ['None', 'FGSM', 'JSMA', 'CWL2', 'LINFPGD', 'LINFBI', 'ENL1', 'ST']
adv_types = ['None', 'FGSM', 'JSMA']
set_of_train_dataset, set_of_test_dataset = [], []
split_percentage = 0.8

for adv_type in adv_types:
    # extract from the store file 
    if adv_type == 'None': fn_name='store/normal.txt'
    else: fn_name='store/'+adv_type+'.txt'
    fp = open(fn_name, 'rb')
    set_of_signatures = pickle.load(fp)
    
    # separate and store in for later training and evaluation 
    split_line = int(len(set_of_signatures)*split_percentage)
    train_set_of_signatures, test_set_of_signatures = set_of_signatures[:split_line], set_of_signatures[split_line:]
    set_of_train_dataset.append(train_set_of_signatures)
    set_of_test_dataset.append(test_set_of_signatures)
    fp.close()
    print(adv_type, len(set_of_signatures), len(train_set_of_signatures), len(test_set_of_signatures))

None 1000 800 200
FGSM 813 650 163
JSMA 966 772 194


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch

guard_model = Guard()
guard_model.train()
epoches = 15

accs, losses = train_guard_model(guard_model, set_of_train_dataset, adv_types, epoches)
losses = [x.item() for x in losses]
guard_model.eval()
torch.save(guard_model, 'store/guard.pt')
# guard_model = torch.load('guard.pt')

benign correct: 798 / 800
adv ( FGSM ) correct: 631 / 650
adv ( JSMA ) correct: 772 / 772
epoch: 1 loss: 99.35013580322266
acc: 0.9905490549054905
benign correct: 776 / 800
adv ( FGSM ) correct: 624 / 650
adv ( JSMA ) correct: 772 / 772
epoch: 2 loss: 309.6373291015625
acc: 0.9774977497749775
benign correct: 785 / 800
adv ( FGSM ) correct: 632 / 650
adv ( JSMA ) correct: 771 / 772
epoch: 3 loss: 113.44163513183594
acc: 0.9846984698469847
benign correct: 788 / 800
adv ( FGSM ) correct: 639 / 650
adv ( JSMA ) correct: 772 / 772
epoch: 4 loss: 143.84078979492188
acc: 0.9896489648964897
benign correct: 774 / 800
adv ( FGSM ) correct: 629 / 650
adv ( JSMA ) correct: 766 / 772
epoch: 5 loss: 202.35244750976562
acc: 0.9761476147614762
benign correct: 757 / 800
adv ( FGSM ) correct: 639 / 650
adv ( JSMA ) correct: 770 / 772
epoch: 6 loss: 303.2123107910156
acc: 0.9747974797479748
benign correct: 772 / 800
adv ( FGSM ) correct: 640 / 650
adv ( JSMA ) correct: 767 / 772
epoch: 7 loss: 171.713775

In [ ]:
import copy
from scipy.ndimage.filters import gaussian_filter1d
smooth_accs = gaussian_filter1d(accs, sigma=2)
smooth_losses = gaussian_filter1d(losses, sigma=2)
norm_smooth_losses = [float(i)/max(smooth_losses) for i in smooth_losses]

from matplotlib import pyplot as plt
%matplotlib inline
# declare a figure
fig = plt.figure()
fig.set_figheight(10)
fig.set_figwidth(15)

# summarize history for accuracy
plt.plot(smooth_accs)
plt.plot(norm_smooth_losses)
plt.title('model accuracy and loss')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['acc', 'loss'], loc='upper left')
plt.show()

In [ ]:
test_guard_model(guard_model, set_of_test_dataset, adv_types)